In [1]:
import pymysql
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine
import yaml

In [2]:
# db정보 가져오기
with open('../yamls/sql_info.yaml') as f:

    info = yaml.load(f, Loader=yaml.FullLoader)

host = info['MARIADB']['IP']
user = info['MARIADB']['USER']
passwd=info['MARIADB']['PASSWD']
db = info['MARIADB']['DB']
port = info['MARIADB']['PORT']

# 선수 테이블 생성하기 

In [3]:
conn = pymysql.connect(host = host, user = user, passwd=passwd, db = db, charset='utf8', port = port,cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()
sql = '''select * from batting_info
'''
cur.execute(sql)

result = cur.fetchall()
batting = pd.DataFrame(result)

sql = '''select * from pitching_info
'''
cur.execute(sql)

result = cur.fetchall()
pitching = pd.DataFrame(result)

conn.close()

In [4]:
# 투수, 타자 df에서 필요한 컬럼만 가져오기
batting = batting[['player_name','player_birth']].drop_duplicates(['player_name','player_birth'])
pitching = pitching[['player_name','player_birth']].drop_duplicates(['player_name','player_birth'])

# 투수, 타자 정보 concat 후 중복제거하기
player = pd.concat([batting,pitching]).drop_duplicates(['player_name','player_birth'])
player = pd.concat([batting,pitching])

# 완성된 데이터 적재 
engine = create_engine(f"mysql+pymysql://{user}:{passwd}@{host}:{port}/{db}?charset=utf8")
conn = engine.connect()
player.to_sql(name = 'player_info', con = engine, if_exists = 'append', index=False)
conn.close()

## week 컬럼생성

In [72]:

conn = pymysql.connect(host = host, user = user, passwd=passwd, db = db, charset='utf8', port = port,cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()
sql = '''select * from batting_info
'''
cur.execute(sql)

result = cur.fetchall()
batting = pd.DataFrame(result)

sql = '''select * from pitching_info
'''
cur.execute(sql)

result = cur.fetchall()
pitching = pd.DataFrame(result)

conn.close()




In [73]:
# isocalendar 모듈을 사용하여 week 계산
batting['week'] = batting['yyyymmdd'].astype(str).apply(lambda x :  x[0:4] + str(datetime.strptime(x[0:4] + '-'+ x[4:6]+'-'+x[6:8] ,'%Y-%m-%d').isocalendar().week))
pitching['week'] = pitching['yyyymmdd'].astype(str).apply(lambda x :  x[0:4] + str(datetime.strptime(x[0:4] + '-'+ x[4:6]+'-'+x[6:8] ,'%Y-%m-%d').isocalendar().week))

# 투수 이닝수 전처리하기 


pitching['IP2'] = pitching['IP'].astype(str).apply(lambda x : int(x.split('.')[1]))
pitching['IP'] = pitching['IP'].astype(str).apply(lambda x : int(x.split('.')[0]))
 

# 주별 합산 지표 컬럼만 추리기
batting_week = batting.groupby(['week','player_name','player_birth','team']).sum()[['TPA','AB','R','H', 'HR','RBI','BB', 'HBP','SO','GO','FO','PIT','GDP','LOB']].reset_index()
pitching_week = pitching.groupby(['week','player_name','player_birth','team']).sum()[['today_type','IP','IP2','TBF','H','R','ER','BB','HBP','K','HR']].reset_index()

In [74]:
# 규정 타석, 이닝을 계산하기위한 주별 팀 경기수 변수 생성
team_games = batting.groupby(['week','team','yyyymmdd']).count().reset_index()
team_games = team_games.groupby(['week','team']).count().reset_index()[['week','team','yyyymmdd']]
team_games['game_count'] = team_games['yyyymmdd']
team_games = team_games[['week','team','game_count']]

In [75]:
batting_week = batting_week.merge(team_games, on = ['week', 'team'], how = 'left')
pitching_week = pitching_week.merge(team_games, on = ['week', 'team'], how = 'left')

In [77]:
batting_week['RTPA'] = batting_week['game_count']*3.1
batting_week['AVG'] = batting_week['H']/batting_week['AB']
batting_week['AVG'] =batting_week['AVG'].fillna(0.0) 
batting_week = batting_week[['week', 'player_name', 'player_birth', 'team', 'TPA','RTPA', 'AB', 'R', 'H',
       'HR', 'RBI', 'BB', 'HBP', 'SO', 'GO', 'FO', 'PIT', 'GDP', 'LOB',
         'AVG','game_count']]


pitching_week['RIP'] = pitching_week['game_count'] * 1.0
 
# 투수 이닝 합산
pitching_week['IP'] = pitching_week['IP'] + pitching_week['IP2'].apply(lambda x : x//3) + pitching_week['IP2'].apply(lambda x : x%3/10) 
pitching_week = pitching_week[['week', 'player_name', 'player_birth', 'team', 'today_type', 'IP', 'RIP','TBF', 'H', 'R', 'ER', 'BB',
       'HBP', 'K', 'HR', 'game_count']]

In [80]:
engine = create_engine(f"mysql+pymysql://{user}:{passwd}@{host}:{port}/{db}?charset=utf8")
conn = engine.connect()
batting_week.to_sql(name = 'weekly_batting_info', con = engine, if_exists = 'append', index=False)
conn.close()

engine = create_engine(f"mysql+pymysql://{user}:{passwd}@{host}:{port}/{db}?charset=utf8")
conn = engine.connect()
pitching_week.to_sql(name = 'weekly_pitching_info', con = engine, if_exists = 'append', index=False)
conn.close()